In [1]:
import os
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from mne.decoding import CSP
import sklearn.feature_selection
from sklearn.feature_selection import mutual_info_classif
from scipy import signal
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import random 
from sklearn.model_selection import train_test_split

random.seed(42)

In [2]:
filePath= '/Users/yaman/Library/CloudStorage/GoogleDrive-s203192@stn.nagaokaut.ac.jp/マイドライブ/実験データ/20230519_zikken'

brainAdress = ['Fp1','AF7','AF3','F1','F3','F5','F7','FT7','FC5',
'FC3','FC1','C1','C3','C5','T7','TP7','CP5','CP3','CP1','P1','P3',
'P5','P7','P9','PO7','PO3','O1','Iz','Oz','POz','Pz','CPz','Fpz','Fp2',
'AF8','AF4','AFz','Fz','F2','F4','F6','F8','FT8','FC6','FC4','FC2',
'FCz','Cz','C2','C4','C6','T8','TP8','CP6','CP4','CP2','P2','P4',
'P6','P8','P10','PO8','PO4','O2']

# dataクラス
class O_Data:
    def __init__(self, eeg, index, label):
        self.eeg = eeg
        self.index = index
        self.label = label
        self.stack = []

    def push(self, item):
        self.stack.append(item)

o_data65_label1 = list()#label1のデータ
o_data = list()
label_data = list()#labelを結合したデータ


All_data = list()  # 全データを結合

def Standardization(data): #標準化
    after_data = scipy.stats.zscore(data)
    #b = np.average(after_data)
    #c = np.var(after_data)
    return after_data

file_name = list()  # すべての.matファイルの名前
for file in os.listdir(filePath):
    base, ext = os.path.splitext(file)
    if ext == '.mat':
        file_name = sorted(file_name)
        file_name.append(file)
print('Filelist')
print(file_name)

Filelist
['10EEG_grasping_19-May-2023_sub3-4.mat', '11EEG_grasping_19-May-2023_sub3-5.mat', '12EEG_grasping_19-May-2023_sub3-6.mat', '1EEG_grasping_19-May-2023_sub1-1.mat', '2EEG_grasping_19-May-2023_sub1-2.mat', '3EEG_grasping_19-May-2023_sub2-1.mat', '4EEG_grasping_19-May-2023_sub2-2.mat', '5EEG_grasping_19-May-2023_sub2-3.mat', '6EEG_grasping_19-May-2023_sub2-4.mat', '8EEG_grasping_19-May-2023_sub3-2.mat', '9EEG_grasping_19-May-2023_sub3-3.mat', '7EEG_grasping_19-May-2023_sub3-1.mat']


In [3]:
#解析するデータを選択
file_name1 = [file_name[5],file_name[6],file_name[7],file_name[8]]
file_num1 = len(file_name1)
trial_num = 30
all_trial = len(file_name1) * trial_num
all_d = 0
CH = 64
#label1を読み込む
for s in range(file_num1):
    Dictionary = scipy.io.loadmat(filePath+"/"+file_name1[s])
    for t in range(trial_num):
        for i, key in enumerate(Dictionary.keys()):
            if i > 2:
                a = Dictionary[key]
                b = a[0, t][0][0]
                eeg = b[0][0:66, :]
                index = b[1]
                samplerate = b[2]
                label = b[5][0, 0]
                o_data = O_Data(eeg, index, label)
                o_data65_label1.append(o_data)
print("image data_loading")

image data_loading


In [4]:
c1 = o_data65_label1[0].eeg[: , 0:10240]
for n in range(len(file_name1)*trial_num-1):  # all_trial
    g1 = o_data65_label1[n+1].eeg[: , 0:10240]
    c1 = np.dstack([c1, g1])
all_data = c1.transpose(2,0,1)
print(np.shape(all_data))
all_data = all_data.astype(np.double)
label_data = list()#labelを結合したデータ
for n in range(all_trial):
    label_data.append(o_data65_label1[n].label)
print(label_data)
print(np.shape(label_data))

(120, 66, 10240)
[1, 2, 1, 2, 2, 3, 2, 1, 3, 1, 3, 2, 1, 3, 3, 3, 2, 1, 1, 2, 3, 2, 3, 1, 1, 3, 2, 2, 3, 1, 2, 1, 3, 2, 3, 3, 3, 1, 1, 2, 1, 3, 2, 1, 3, 3, 2, 3, 1, 2, 2, 1, 1, 1, 3, 3, 2, 2, 2, 1, 1, 2, 3, 3, 1, 2, 2, 1, 1, 3, 1, 1, 3, 3, 3, 2, 3, 2, 2, 3, 2, 1, 3, 3, 2, 1, 1, 2, 1, 2, 3, 3, 3, 1, 2, 2, 3, 2, 2, 3, 1, 2, 2, 2, 3, 1, 1, 1, 2, 1, 1, 3, 3, 3, 2, 2, 1, 1, 3, 1]
(120,)


In [5]:
#numpy配列の定義
task1 = o_data65_label1[0].eeg[: , 0:10240]
task2 = o_data65_label1[0].eeg[: , 0:10240]
task3 = o_data65_label1[0].eeg[: , 0:10240]
for n in range(all_trial):
    if label_data[n] == 1:
        join = o_data65_label1[n].eeg[: , 0:10240]
        task1 = np.dstack([task1, join])
    if label_data[n] == 2:
        join = o_data65_label1[n].eeg[: , 0:10240]
        task2 = np.dstack([task2, join])
    if label_data[n] == 3:
        join = o_data65_label1[n].eeg[: , 0:10240]
        task3 = np.dstack([task3, join])
#task1低速度
task1_data = task1.transpose(2,0,1)
task1_data = task1_data.astype(np.double)

task1_data = np.delete(task1_data,0,axis=0)
print(np.shape(task1_data))
#task2低速度
task2_data = task2.transpose(2,0,1)
task2_data = task2_data.astype(np.double)
task2_data = np.delete(task2_data,0,axis=0)
print(np.shape(task2_data))
#task3想起なし
task3_data = task3.transpose(2,0,1)
task3_data = task3_data.astype(np.double)
task3_data = np.delete(task3_data,0,axis=0)
print(np.shape(task2_data))

(40, 66, 10240)
(40, 66, 10240)
(40, 66, 10240)


In [6]:
b,a=signal.butter(8,[(16/1024),(64/1024)],'bandpass')
for i in range(int(all_trial/3)):
    for j in range(64):
        #バンドパス #resampling
        task1_data[i][j][:] = signal.filtfilt(b,a,task1_data[i][j][:])
        task2_data[i][j][:] = signal.filtfilt(b,a,task2_data[i][j][:])
        task3_data[i][j][:] = signal.filtfilt(b,a,task3_data[i][j][:])

In [7]:
def butter_bandpass(lowcut,highcut,fs,order):
    nyq=0.5*fs
    low=lowcut/nyq
    high=highcut/nyq
    b,a==signal.butter(8,[low,high],'bandpass')
    return b,a
def butter_bandpass_filter(data,lowcut,highcut,fs,order):
    b,a=butter_bandpass(lowcut,highcut,fs,order)
    y=signal.filtfilt(b,a,data,axis=2)
    return y

In [8]:
class_data = np.concatenate([task1_data[:,0:64,4*1024:10*1024],task2_data[:,0:64,4*1024:10*1024]])
class_label = np.zeros(80,dtype=int)
class_label[40:] = 1

In [9]:
csp=CSP(n_components=4, reg=None, log=True, norm_trace=False)

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(class_data, class_label, test_size=0.6)


In [14]:
np.shape(X_train)

(32, 64, 6144)

In [15]:
X_train

array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       ...,

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan

In [13]:
features_train=[]
features_test=[]
freq=[8,12,16,20,24,28,32]
for freq_count in range(len(freq)):
#loop for freqency
    lower=freq[freq_count]
    if lower==freq[-1]:
        break
    higher=freq[freq_count+1]
    X_train_filt=butter_bandpass_filter(X_train,lowcut=lower,highcut=higher,fs=1024,order=8)
    X_test_filt=butter_bandpass_filter(X_test,lowcut=lower,highcut=higher,fs=1024,order=8)
    tmp_train=csp.fit_transform(X_train_filt,Y_train)
    tmp_test=csp.transform(X_test_filt)
    if freq_count==0:
        features_train=tmp_train
        features_test=tmp_test
    else:
        features_train=np.concatenate((features_train,tmp_train),axis=1)
        features_test=np.concatenate((features_test,tmp_test),axis=1)
print(features_train.shape)
print(features_test.shape)

Computing rank from data with rank=None


ValueError: array must not contain infs or NaNs

In [ ]:
select_K=sklearn.feature_selection.SelectKBest(mutual_info_classif,k=10).fit(features_train,Y_train)
New_train=select_K.transform(features_train)
#np.random.shuffle(New_train)
New_test=select_K.transform(features_test)
#np.random.shuffle(New_test)
print(New_train.shape)
print(New_test.shape)
ss = preprocessing.StandardScaler()
X_select_train = ss.fit_transform(New_train,Y_train)
X_select_test = ss.fit_transform(New_test)

In [ ]:
from sklearn.svm import SVC
clf=svm.SVC(C=0.8,kernel='rbf')
clf.fit(X_select_train,Y_train)
y_pred=clf.predict(X_select_test)
print(Y_test)
print(y_pred)
acc=accuracy_score(Y_test,y_pred)
print(acc)